# Image recognition using MINST dataset with class model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [5]:
class DigitsRecogniserModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 7, kernel_size=3, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(7, 16, kernel_size=2, stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = x.view(-1, 16 * 6 * 6)  # Adjust to match flattened size
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# Test the model dimensions
test_input = torch.randn(1, 1, 28, 28)  # Batch size 1, 1 channel, 28x28 image
model = DigitsRecogniserModel()
output = model(test_input)
print("Output shape:", output.shape)

Output shape: torch.Size([1, 10])


In [7]:
transform = transforms.ToTensor() # Convert the datatset to 4-dimenentional

In [8]:
training_dataset = datasets.MNIST(root='/digits_data', train=True, download=True, transform=transform) 
testing_dataset = datasets.MNIST(root='/digits_data', train=False, download=True, transform=transform) 

In [9]:
train_loader = DataLoader(dataset=training_dataset, batch_size=5, shuffle=True)
test_loader = DataLoader(dataset=training_dataset, batch_size=5, shuffle=False)

In [10]:
training_dataset, testing_dataset

(Dataset MNIST
     Number of datapoints: 60000
     Root location: /digits_data
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: /digits_data
     Split: Test
     StandardTransform
 Transform: ToTensor())

In [11]:
model = DigitsRecogniserModel()
model

DigitsRecogniserModel(
  (conv1): Conv2d(1, 7, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(7, 16, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [12]:
# declare loss function and optimiser to calculate loss and optimize data
loss_function = nn.CrossEntropyLoss() # add a loss function
optimiser = torch.optim.Adam(model.parameters(), lr=0.01) # add optimser

In [13]:
# tEST FOR DIMENTIONS Mathc for model
test_input = torch.randn(1, 1, 28, 28)  # Batch size = 1, channels = 1, image size = 28x28
output = model(test_input)
print(output.size())


torch.Size([1, 10])


In [14]:

epochs = 6
train_losses = []
test_losses = []

for epoch in range(epochs):

    # Train
    for batch, (X_train, y_train) in enumerate(train_loader):
        batch += 1 # add 1 to batch to start count from 1, initially is be 0
        y_pred = model(X_train) # train model
        loss = loss_function(y_pred, y_train)

        # add train loss to train_losses
        train_losses.append(loss)

        #print training state parameters in every 500 batch
        if batch % 500 == 0:
            print(f" Epoch : {epoch}\t Batch : {batch}\t Loss : {loss.item()}")
        #optimize model parameter
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    # Test
    with torch.no_grad():
        for batch, (X_test, y_test) in enumerate(test_loader):
            y_eval = model(X_test) # test model
            loss = loss_function(y_eval, y_test) # add loss function

            
            # add train loss to train_losses
            test_losses.append(loss)

 Epoch : 0	 Batch : 500	 Loss : 0.867453932762146
 Epoch : 0	 Batch : 1000	 Loss : 1.0533723831176758
 Epoch : 0	 Batch : 1500	 Loss : 0.010403141379356384
 Epoch : 0	 Batch : 2000	 Loss : 0.011599701829254627
 Epoch : 0	 Batch : 2500	 Loss : 0.1807120144367218
 Epoch : 0	 Batch : 3000	 Loss : 0.21489953994750977
 Epoch : 0	 Batch : 3500	 Loss : 0.43504539132118225
 Epoch : 0	 Batch : 4000	 Loss : 1.0761748552322388
 Epoch : 0	 Batch : 4500	 Loss : 1.0951368808746338
 Epoch : 0	 Batch : 5000	 Loss : 0.9545404314994812
 Epoch : 0	 Batch : 5500	 Loss : 0.0018839600961655378
 Epoch : 0	 Batch : 6000	 Loss : 7.986945092852693e-06
 Epoch : 0	 Batch : 6500	 Loss : 0.16693586111068726
 Epoch : 0	 Batch : 7000	 Loss : 0.011940227821469307
 Epoch : 0	 Batch : 7500	 Loss : 1.839482069015503
 Epoch : 0	 Batch : 8000	 Loss : 1.4149731397628784
 Epoch : 0	 Batch : 8500	 Loss : 0.10825206339359283
 Epoch : 0	 Batch : 9000	 Loss : 1.5358644723892212
 Epoch : 0	 Batch : 9500	 Loss : 0.0266995914280414

In [1]:
train_losses = [i.item() for i in train_losses]
test_losses = [i.item() for i in test_losses]
plt.plot(test_losses)
plt.show()

NameError: name 'train_losses' is not defined

In [20]:
train_losses[0].item()

2.2472662925720215